In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy
import math
from sklearn.model_selection import train_test_split
from scipy.sparse.linalg import svds

In [7]:
articles_df=pd.read_csv("/content/drive/MyDrive/shared_articles.csv")

In [9]:
articles_df=articles_df[articles_df.eventType=="CONTENT SHARED"]

In [11]:
interactions_df=pd.read_csv("/content/drive/MyDrive/users_interactions.csv")

In [29]:
users_interactions_count_df=interactions_df.groupby(['personId','contentId']).size().groupby('personId').size()

In [52]:
users_with_enough_interactions_df=users_interactions_count_df[users_interactions_count_df>=5].reset_index()[['personId']]
users_with_enough_interactions_df

,personId
0,-9223121837663643404
1,-9212075797126931087
2,-9207251133131336884
3,-9199575329909162940
4,-9196668942822132778
...,...
1135,9165571805999894845
1136,9187866633451383747
1137,9191849144618614467
1138,9199170757466086545


In [68]:
interactions_from_selected_users_df=pd.merge(interactions_df,users_with_enough_interactions_df, left_on = 'personId',
               right_on = 'personId',how='right')

In [69]:
interactions_from_selected_users_df

,timestamp,eventType,contentId,personId,sessionId,userAgent,userRegion,userCountry
0,1463138398,VIEW,7516228655554309785,-9223121837663643404,-4482197405545551645,Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKi...,SP,BR
1,1463656314,VIEW,3041906492387035176,-9223121837663643404,-7824685088995468735,Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKi...,SP,BR
2,1464113091,VIEW,-3750879736572068916,-9223121837663643404,-2774275024909061125,Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKi...,SP,BR
3,1462283851,VIEW,-730957269757756529,-9223121837663643404,2625340673871268120,Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKi...,SP,BR
4,1462452127,VIEW,-8949113594875411859,-9223121837663643404,-3673331845456357462,Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKi...,SP,BR
...,...,...,...,...,...,...,...,...
69863,1482932930,VIEW,-1572252285162838958,9210530975708218054,-7673376373070890422,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,SP,BR
69864,1484126240,VIEW,3660989387512978561,9210530975708218054,1216354488778990261,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,SP,BR
69865,1483015651,VIEW,-1572252285162838958,9210530975708218054,-2172456475209001768,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,SP,BR
69866,1483015617,VIEW,-1572252285162838958,9210530975708218054,-2172456475209001768,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,SP,BR


In [74]:
event_type_strength = {
   'VIEW': 1.0,
   'LIKE': 2.0, 
   'BOOKMARK': 2.5, 
   'FOLLOW': 3.0,
   'COMMENT CREATED': 4.0,  
}

In [75]:
interactions_from_selected_users_df['eventStrength']=interactions_from_selected_users_df.eventType.apply(lambda x:event_type_strength[x])

In [82]:
interactions_from_selected_users_df.shape

(69868, 9)

In [81]:
interactions_from_selected_users_df[interactions_from_selected_users_df['personId']==-9207251133131336884]

,timestamp,eventType,contentId,personId,sessionId,userAgent,userRegion,userCountry,eventStrength
72,1467140430,VIEW,7521700660190108859,-9207251133131336884,-315356171277800889,NaN,NaN,NaN,1.0
73,1467140430,VIEW,7521700660190108859,-9207251133131336884,-315356171277800889,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,SP,BR,1.0
74,1467140748,LIKE,7521700660190108859,-9207251133131336884,-315356171277800889,NaN,NaN,NaN,2.0
75,1464123542,VIEW,-8742648016180281673,-9207251133131336884,1914439309863179675,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,SP,BR,1.0
76,1464123617,LIKE,-8742648016180281673,-9207251133131336884,1914439309863179675,NaN,NaN,NaN,2.0
77,1467719472,VIEW,-969155230116728853,-9207251133131336884,8538984087220668594,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,SP,BR,1.0
78,1469213710,LIKE,-4434534460030275781,-9207251133131336884,6009793495426177010,NaN,NaN,NaN,2.0
79,1467695437,LIKE,-9216926795620865886,-9207251133131336884,8538984087220668594,NaN,NaN,NaN,2.0
80,1469212069,VIEW,-4434534460030275781,-9207251133131336884,6009793495426177010,Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebK...,SP,BR,1.0
81,1467689754,VIEW,-9216926795620865886,-9207251133131336884,8538984087220668594,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,SP,BR,1.0


In [83]:
def smooth_user_preference(x):
    return math.log(1+x, 2)

In [99]:
interactions_full_df=interactions_from_selected_users_df.groupby(['personId','contentId'])["eventStrength"].sum().apply(smooth_user_preference).reset_index()

In [101]:
interactions_full_df[interactions_full_df['contentId']==-7423191370472335463]

,personId,contentId,eventStrength
4,-9223121837663643404,-7423191370472335463,3.169925
1371,-8830250090736356260,-7423191370472335463,1.000000
1538,-8763398617720485024,-7423191370472335463,1.000000
2736,-8205402408645015051,-7423191370472335463,1.000000
3387,-7658896905823060250,-7423191370472335463,2.000000
3400,-7656701953949775820,-7423191370472335463,1.000000
4025,-7365517474830766511,-7423191370472335463,1.000000
4142,-7267769888748948232,-7423191370472335463,1.000000
4306,-7240735065448254654,-7423191370472335463,3.857981
5219,-6805567438224540711,-7423191370472335463,1.000000


In [102]:
interactions_train_df, interactions_test_df = train_test_split(interactions_full_df,
                                   stratify=interactions_full_df['personId'], 
                                   test_size=0.20,
                                   random_state=42)

In [103]:
print('# interactions on Train set: %d' % len(interactions_train_df))
print('# interactions on Test set: %d' % len(interactions_test_df))

# interactions on Train set: 31284
# interactions on Test set: 7822


In [105]:
users_items_pivot_matrix_df=interactions_train_df.pivot(index='personId',columns='contentId',values='eventStrength').fillna(0)

In [114]:
users_items_pivot_matrix = users_items_pivot_matrix_df.values
users_ids = list(users_items_pivot_matrix_df.index)
users_items_pivot_matrix.shape

(1140, 2926)

In [115]:
NUMBER_OF_FACTORS_MF=15
U,sigma,Vt=svds(users_items_pivot_matrix, k = NUMBER_OF_FACTORS_MF)

In [118]:
sigma = np.diag(sigma)
sigma.shape

(15, 15)

In [119]:
Vt.shape

(15, 2926)

In [120]:
all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) 
cf_preds_df = pd.DataFrame(all_user_predicted_ratings, columns = users_items_pivot_matrix_df.columns, index=users_ids).transpose()


In [121]:
cf_preds_df

,-9223121837663643404,-9212075797126931087,-9207251133131336884,-9199575329909162940,-9196668942822132778,-9188188261933657343,-9172914609055320039,-9156344805277471150,-9120685872592674274,-9109785559521267180,-9063420486253202900,-9060214117327732109,-9047547311469006438,-9016528795238256703,-9009798162809551896,-9001583565812478106,-8994220765455693336,-8909668725653743114,-8891033171626175843,-8860671864164757449,-8854674432071487111,-8853658195208337106,-8845298781299428018,-8830250090736356260,-8823950498314351783,-8802075878443651241,-8784674845716296727,-8781635134606732409,-8781306637602263252,-8763398617720485024,-8738496712327699923,-8719462623048086192,-8704807962619440953,-8699750646678621887,-8694104221113176052,-8686631410634491662,-8674958742744576254,-8672331451814079632,-8670749047273764903,-8652741825481604192,...,8791271631167250981,8801420707973230165,8801970869404590779,8813266398846460512,8841741572929644986,8847054836611412804,8855523843512271162,8862260182894039021,8872819156169667456,8874741321583329336,8879844298911979276,8892482595912468268,8907499588729810535,8913362709216003291,8920667914865172372,8940614478925413056,8941502917401491878,8961723342122872302,8965285988346645117,8968131284214320024,8982783231149017560,8992729171160464416,9013651444868609421,9033898219489253274,9037410398700100618,9038446466275805109,9050204922960952289,9090527742744334314,9091970136990402395,9102085903669288476,9105269044962898535,9109075639526981934,9135582630122950040,9137372837662939523,9148269800512008413,9165571805999894845,9187866633451383747,9191849144618614467,9199170757466086545,9210530975708218054
contentId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
-9222795471790223670,0.010399,-0.000193,-0.012547,0.052958,-0.010035,-0.005412,-0.003918,0.049302,-0.013458,-0.020052,-0.009529,0.001567,-0.012044,-0.065134,0.112747,0.041108,-0.010481,-0.005297,-0.034884,-0.123017,-0.008897,-0.042487,0.416457,-0.033627,-0.003643,-0.000428,-0.007387,-0.007513,0.006235,0.058278,-0.028610,0.042954,0.000090,-0.008141,0.134043,-0.003323,0.032243,-0.033136,-0.039186,0.001904,...,0.000177,0.020430,-0.018576,0.023053,-0.024097,-0.007354,0.013645,-0.018282,-0.014919,-0.009511,-0.019433,0.000082,-0.003258,-0.002602,0.004572,-0.017065,0.001240,-0.001879,-0.002489,0.510757,-0.007451,-0.018229,-0.004900,-0.009748,0.002856,0.003318,-0.007141,-0.006217,0.009393,0.030966,-0.005306,-0.089463,-0.000048,0.015023,0.020953,-0.009394,-0.019114,-0.029954,-0.018455,-0.015064
-9216926795620865886,0.000819,-0.000313,0.006595,-0.000649,0.000151,0.000338,0.000197,0.000861,0.002341,0.001765,0.000693,-0.000401,-0.001323,0.016495,0.009792,0.005556,0.004797,0.001999,0.008558,0.036923,0.005375,0.019304,-0.009210,-0.000040,0.000914,0.000683,0.001836,0.000193,0.003247,0.009282,0.001267,0.003316,-0.000087,0.007292,0.001487,0.000276,0.009248,0.006766,0.009996,-0.000612,...,0.000489,0.001326,0.000265,0.002428,0.002844,-0.001767,0.003285,0.004587,0.005100,-0.000124,0.004198,0.001862,0.001347,-0.000930,0.016121,0.003219,0.001562,0.002164,0.000682,0.024945,-0.000414,0.002806,-0.000472,0.002431,-0.000802,0.002470,0.000775,-0.000109,0.001362,-0.000543,0.000095,0.013915,0.000506,0.001460,0.014863,0.000940,0.002391,0.008057,0.004670,0.003277
-9194572880052200111,-0.017253,-0.002646,-0.005907,-0.003666,0.021300,-0.001278,0.029100,-0.014730,-0.023410,0.004950,0.025522,0.006706,0.016231,-0.085721,0.058917,0.029974,-0.002729,0.012110,-0.002258,0.015465,-0.002322,0.162668,-0.110257,0.043068,0.003892,0.002378,0.020120,-0.000989,0.095824,0.079587,-0.014877,0.033582,0.004351,0.023526,-0.057510,0.005366,0.082665,0.016139,0.079913,0.022207,...,-0.000575,0.013275,-0.005904,-0.010228,0.013855,0.016897,0.077516,-0.003531,0.044262,0.015127,0.057267,0.032941,0.011972,0.009667,0.005471,-0.022793,0.011801,0.001594,-0.002086,0.106361,0.000542,0.012707,-0.007648,-0.001696,-0.001347,0.020591,0.005007,0.001418,0.016003,-0.010067,0.011529,0.046012,0.010485

In [122]:
def get_items_interacted(person_id, interactions_df):
    interacted_items = interactions_df.loc[person_id]['contentId']
    return set(interacted_items if type(interacted_items) == pd.Series else [interacted_items])

In [123]:
interactions_train_indexed_df = interactions_train_df.set_index('personId')


In [138]:
interactions_train_indexed_df

,contentId,eventStrength
personId,,
3609194402293569455,-3482581255202910790,1.000000
3938645257702379823,-8159730897893673528,1.000000
-8417075466521236595,-6783772548752091658,1.000000
-8399605302938582500,-3716447017462787559,1.000000
-1684566741313744330,3367026768872537336,1.000000
...,...,...
-2050699458865052139,-993224631915984347,1.000000
-1836083230511905974,6797033574746412211,1.584963
-292717837486008213,-5386512752101672374,1.000000


In [135]:
get_items_interacted(-9223121837663643404, interactions_train_indexed_df)

{-8377626164558006982,
 -8187220755213888616,
 -7423191370472335463,
 -7331393944609614247,
 -6872546942144599345,
 -6590819806697898649,
 -6558712014192834002,
 -6545872007932025533,
 -6484638837208285334,
 -5002383425685129595,
 -4541461982704074404,
 -4233177915193302509,
 -4205346868684833897,
 -3912939678517879962,
 -3750879736572068916,
 -730957269757756529,
 -559964548932224920,
 -447851796385928420,
 834896074125772354,
 921770761777842242,
 943818026930898372,
 1436883058900979473,
 3041906492387035176,
 3180828616327439381,
 3367026768872537336,
 4419562057180692966,
 4563606877148407012,
 5087084654882097891,
 5211673327552264703,
 5293701842202310496,
 6615662748793536194,
 7229629480273331039,
 7516228655554309785,
 7734121175534200554}

In [154]:
def recommend_items( cf_preds_df,articles_df,user_id, items_to_ignore=[], topn=10, verbose=False):
        # Get and sort the user's predictions
        sorted_user_predictions = cf_preds_df[user_id].sort_values(ascending=False).reset_index().rename(columns={user_id: 'recStrength'})

        # Recommend the highest predicted rating content that the user hasn't seen yet.
        recommendations_df = sorted_user_predictions[~sorted_user_predictions['contentId'].isin(items_to_ignore)].sort_values('recStrength', ascending = False).head(topn)


        recommendations_df = recommendations_df.merge(articles_df, how = 'left', 
                                                          left_on = 'contentId', 
                                                          right_on = 'contentId')[['recStrength', 'contentId', 'title', 'url', 'lang']]


        return recommendations_df

In [132]:
cf_preds_df[-9223121837663643404].sort_values(ascending=False).reset_index().rename(columns={-9223121837663643404: 'recStrength'})

,contentId,recStrength
0,-8208801367848627943,0.397180
1,943818026930898372,0.281611
2,3149164017776669829,0.254751
3,2072448887839540892,0.250393
4,-4029704725707465084,0.227099
...,...,...
2921,3818189513627822856,-0.055110
2922,-5161313513317812402,-0.056386
2923,2719909253419802298,-0.058613
2924,-6999287066519531005,-0.060424


In [136]:
interactions_df

,timestamp,eventType,contentId,personId,sessionId,userAgent,userRegion,userCountry
0,1465413032,VIEW,-3499919498720038879,-8845298781299428018,1264196770339959068,NaN,NaN,NaN
1,1465412560,VIEW,8890720798209849691,-1032019229384696495,3621737643587579081,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2...,NY,US
2,1465416190,VIEW,310515487419366995,-1130272294246983140,2631864456530402479,NaN,NaN,NaN
3,1465413895,FOLLOW,310515487419366995,344280948527967603,-3167637573980064150,NaN,NaN,NaN
4,1465412290,VIEW,-7820640624231356730,-445337111692715325,5611481178424124714,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
72307,1485190425,LIKE,-6590819806697898649,-9016528795238256703,8614469745607949425,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_4...,MG,BR
72308,1485190425,VIEW,-5813211845057621660,102305705598210278,5527770709392883642,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,SP,BR
72309,1485190072,VIEW,-1999468346928419252,-9196668942822132778,-8300596454915870873,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,SP,BR
72310,1485190434,VIEW,-6590819806697898649,-9016528795238256703,8614469745607949425,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_4...,MG,BR


In [145]:
person_id=-9223121837663643404

In [157]:
recommended=recommend_items(articles_df=articles_df,cf_preds_df=cf_preds_df,user_id=person_id, items_to_ignore=get_items_interacted(person_id, interactions_train_indexed_df),topn=100000)

In [158]:
recommended.contentId

0      -8208801367848627943
1       3149164017776669829
2       2072448887839540892
3      -4029704725707465084
4       5238119115012015307
               ...         
2887    3818189513627822856
2888   -5161313513317812402
2889    2719909253419802298
2890   -6999287066519531005
2891    -330801551666885085
Name: contentId, Length: 2892, dtype: int64

In [176]:
visited=(get_items_interacted(person_id, interactions_train_indexed_df))

In [172]:
a=(interactions_train_indexed_df.loc[3609194402293569455]["contentId"])

In [179]:
visited

{-8377626164558006982,
 -8187220755213888616,
 -7423191370472335463,
 -7331393944609614247,
 -6872546942144599345,
 -6590819806697898649,
 -6558712014192834002,
 -6545872007932025533,
 -6484638837208285334,
 -5002383425685129595,
 -4541461982704074404,
 -4233177915193302509,
 -4205346868684833897,
 -3912939678517879962,
 -3750879736572068916,
 -730957269757756529,
 -559964548932224920,
 -447851796385928420,
 834896074125772354,
 921770761777842242,
 943818026930898372,
 1436883058900979473,
 3041906492387035176,
 3180828616327439381,
 3367026768872537336,
 4419562057180692966,
 4563606877148407012,
 5087084654882097891,
 5211673327552264703,
 5293701842202310496,
 6615662748793536194,
 7229629480273331039,
 7516228655554309785,
 7734121175534200554}

In [185]:
recommended_Set=set(recommended.contentId)

In [188]:
recommended_Set.intersection(visited)

set()